## Training Test Notebook
Look at how to better work out training for Mask RCNN models

In [9]:
# imports
import os, sys
import json
from datetime import date
import random

sys.path.insert(1, os.path.abspath(".."))
sys.path.insert(1, os.path.abspath(os.path.join('..', 'pytv')))
from pytv.engine import train_one_epoch, evaluate
import pytv.utils as utils

from data import MaskRCNNDataset
from train import make_new_maskr

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [7]:
# basic setup torch device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using cuda")
else:
    device = torch.device("cpu")
    pring("using cpu")
# setup datasets
DATA_PATH = os.path.join("..", "..", "supp", "testkc")
dta_trn = MaskRCNNDataset(DATA_PATH, "train", None)
train_data = torch.utils.data.DataLoader(dta_trn, batch_size=2, shuffle=True,
                                        num_workers=1, collate_fn=utils.collate_fn)
dta_tst = MaskRCNNDataset(DATA_PATH, "test", None)
test_data = torch.utils.data.DataLoader(dta_tst, batch_size=2, shuffle=False,
                                       num_workers=1, collate_fn=utils.collate_fn)

# make network
net = make_new_maskr(n_class=2, hidden_layer=256)
net = net.to(device)

# setup optimizer
params = [p for p in net.parameters() if p.requires_grad]
optim = torch.optim.SGD(params, lr=0.005, momentum=0.9,
                       weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=3, gamma=0.1)

using cuda


In [10]:
# do example training epoch
metlog = train_one_epoch(net, optim, train_data, device, 0, print_freq=1)

/home/mb46/.conda/envs/pyt/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0]  [0/2]  eta: 0:00:09  lr: 0.005000  loss: 12.9026 (12.9026)  loss_classifier: 0.7161 (0.7161)  loss_box_reg: 0.5925 (0.5925)  loss_mask: 2.6938 (2.6938)  loss_objectness: 8.2721 (8.2721)  loss_rpn_box_reg: 0.6282 (0.6282)  time: 4.5424  data: 1.9638  max mem: 4676
Epoch: [0]  [1/2]  eta: 0:00:02  lr: 0.005000  loss: 12.7793 (12.8410)  loss_classifier: 0.6934 (0.7047)  loss_box_reg: 0.5915 (0.5920)  loss_mask: 2.5784 (2.6361)  loss_objectness: 8.2320 (8.2520)  loss_rpn_box_reg: 0.6282 (0.6561)  time: 2.3639  data: 0.9884  max mem: 5221
Epoch: [0] Total time: 0:00:04 (2.3849 s / it)


In [26]:
{k:v.value for k, v in zip(metlog.meters.keys(), metlog.meters.values())}

{'lr': 0.005,
 'loss': 12.779293060302734,
 'loss_classifier': 0.693372368812561,
 'loss_box_reg': 0.5914664268493652,
 'loss_mask': 2.5783638954162598,
 'loss_objectness': 8.232001304626465,
 'loss_rpn_box_reg': 0.6840900778770447}

In [27]:
coco = evaluate(net, test_data, device=device)

creating index...
index created!
Test:  [0/1]  eta: 0:00:02  model_time: 0.0955 (0.0955)  evaluator_time: 2.4437 (2.4437)  time: 2.8053  data: 0.2656  max mem: 5221
Test: Total time: 0:00:02 (2.8419 s / it)
Averaged stats: model_time: 0.0955 (0.0955)  evaluator_time: 2.4437 (2.4437)
Accumulating evaluation results...
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.050
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50